In [10]:
import pandas as pd
import numpy as np

In [11]:
stores = pd.read_csv("../../own_data/stores.csv")
busstops = pd.read_csv("../../own_data/busstops_norway_with_count.csv")
age_dist = pd.read_csv("../../data/grunnkrets_age_distribution.csv")
households = pd.read_csv("../../data/grunnkrets_households_num_persons.csv")
income = pd.read_csv("../../data/grunnkrets_income_households.csv")
grunnkrets = pd.read_csv("../../data/grunnkrets_norway_stripped.csv")
plaace = pd.read_csv("../../data/plaace_hierarchy.csv")



In [12]:
grunnkrets.drop(columns=['grunnkrets_name', 'geometry'], inplace=True)

In [13]:
grunnkrets = grunnkrets.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')

In [14]:
# remove 2015 data and year column
households = households.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
income = income.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
age_dist = age_dist.sort_values(by=["grunnkrets_id", "year"]).drop_duplicates(subset=["grunnkrets_id"], keep='last')
grunnkrets.drop(columns='year', inplace=True)
households.drop(columns='year', inplace=True)
income.drop(columns='year', inplace=True)
age_dist.drop(columns='year', inplace=True)

In [15]:
# set index to grunnkrets_id for merging
households.set_index('grunnkrets_id', inplace=True)
grunnkrets.set_index('grunnkrets_id', inplace=True)
income.set_index('grunnkrets_id', inplace=True)
age_dist.set_index('grunnkrets_id', inplace=True)

In [16]:
busstops.drop(columns=['geometry', 'Unnamed: 0'], inplace=True)

In [17]:
stores = stores.rename(columns={"Unnamed: 0": "dataset", "Unnamed: 1": "range_index"}).set_index(["dataset", "range_index"])
stores.drop(columns='sales_channel_name', inplace=True)
stores['copy_index'] = stores.index


In [18]:
all_merged = stores.merge(grunnkrets, left_on='grunnkrets_id', right_index=True) \
                   .merge(households, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(income, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(age_dist, left_on='grunnkrets_id', right_index=True, how="left") \
                   .merge(plaace, on='plaace_hierarchy_id') \
                   .merge(busstops, on='busstop_id')

In [19]:
all_merged.index = pd.MultiIndex.from_tuples(
    all_merged['copy_index'], 
    names=['dataset', 'range_index'])

all_merged.drop(columns=['copy_index', 'year', 'plaace_hierarchy_id', 'sales_channel_name', 'lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc'], inplace=True)

In [20]:
all_merged.to_csv("../../own_data/all_merged.csv")